# Code for fitting one logistic regression model to all of the data

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (precision_recall_curve, roc_auc_score, roc_curve)
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import tqdm, itertools
from typing import List, Optional
import matplotlib.pyplot as plt
import pandas as pd, numpy as np, seaborn as sns
from sklearn.linear_model import LogisticRegression
from pathlib import Path
import pandas as pd

In [ ]:
# Data

resultdir = "" # change to your resultdir
results = pd.read_table(resultdir+"/homodimers_logreg_features.tsv", keep_default_na=False) # 922 ids

In [3]:
# Features

fs_opt_feat = [["total_interaction_area"],["structural_consensus"],["best_plddt_max"],["best_pae_min"],["hm_frac_tm0.8"],["multimer_frac_tm0.8"],["hm_frac_tm0.9"],["multimer_frac_tm0.6"]]

nofs_opt_feat = [["max_iptm"],["avg_iptm"],["num_unique_contacts"],["best_plddt_max"],["best_pae_min"]]

In [4]:
def tpr_at_exact_fpr_sklearn(y_true, y_score, target_fpr=0.05, sample_weight=None):
    """
    Returns:
      tpr_exact  : TPR at exactly target_fpr via linear interpolation on ROC
      fpr_near   : nearest realized FPR on the ROC (for tolerance checks)
      tpr_near   : TPR at that nearest realized FPR
    """
    y_true  = np.asarray(y_true).astype(int)
    y_score = np.asarray(y_score).astype(float)
    fpr, tpr, _ = roc_curve(y_true, y_score, sample_weight=sample_weight, drop_intermediate=False)

    # Exact TPR by interpolation (threshold-free)
    tpr_exact = float(np.interp(target_fpr, fpr, tpr))

    # Nearest realized ROC step (for band enforcement/reporting)
    idx = int(np.argmin(np.abs(fpr - target_fpr)))
    fpr_near = float(fpr[idx])
    tpr_near = float(tpr[idx])

    return tpr_exact, fpr_near, tpr_near

def evaluate(y_true, y_score):
    prec, rec, _ = precision_recall_curve(y_true, y_score)
    tpr_5, _, _ = tpr_at_exact_fpr_sklearn(y_true, y_score, 0.05)
    tpr_1, _, _ = tpr_at_exact_fpr_sklearn(y_true, y_score, 0.01)
    return {
        "tpr_at_fpr5": tpr_5, 
        "tpr_at_fpr1": tpr_1, 
        }

def run_logreg(feature_list, input_df):
    df = input_df.copy()
    labels = df["correct_result"].values

    # --- Normalize + flatten feature names ---
    flat_features = []
    for f in feature_list:
        if isinstance(f, (list, tuple, np.ndarray)):
            for g in f:
                if isinstance(g, tuple) and len(g) == 1:
                    flat_features.append(g[0])
                else:
                    flat_features.append(g)
        else:
            flat_features.append(f)

    norm_features = [str(x) for x in flat_features]

    # (same sanity check as before if you want, or reuse your normalize_feature_list)
    cols = set(df.columns)
    missing = [f for f in norm_features if f not in cols]
    if missing:
        raise ValueError(
            "The following requested features are not in df.columns:\n"
            f"{sorted(set(missing))}\n\n"
            f"Available columns include (first 30):\n{list(df.columns)[:30]}"
        )

    # ⬇️ KEY LINE: keep X as a DataFrame
    X = df[norm_features]

    pipe = make_pipeline(
        StandardScaler(),
        LogisticRegression(max_iter=1000, solver="lbfgs", n_jobs=1, random_state=42),
    )
    pipe.fit(X, labels)

    proba = pipe.predict_proba(X)[:, 1]

    # ... rest of the function unchanged ...
    per_pdb_rows = []
    pdb_ids = df["clean_query"].values
    for pdb, yt, ys in zip(pdb_ids, labels, proba):
        per_pdb_rows.append({
            "pdbid": pdb,
            "y_true": int(yt),
            "y_proba": float(ys),
        })

    per_pdb_df = pd.DataFrame(per_pdb_rows)
    return per_pdb_df, pipe




In [5]:
nofseek_df, nofseek_model  = run_logreg(nofs_opt_feat, results)
fseek_df,fseek_model = run_logreg(fs_opt_feat, results)

In [6]:
# Weight of each feature:

import numpy as np
import pandas as pd

def normalize_feature_list(feature_list, df):
    """Flatten + normalize features in the same way as run_logreg."""
    flat_features = []
    for f in feature_list:
        if isinstance(f, (list, tuple, np.ndarray)):
            for g in f:
                if isinstance(g, tuple) and len(g) == 1:
                    flat_features.append(g[0])
                else:
                    flat_features.append(g)
        else:
            flat_features.append(f)

    norm_features = [str(x) for x in flat_features]

    # (Optional) sanity check – same as in run_logreg
    cols = set(df.columns)
    missing = [f for f in norm_features if f not in cols]
    if missing:
        raise ValueError(
            "The following requested features are not in df.columns:\n"
            f"{sorted(set(missing))}\n\n"
            f"Available columns include (first 30):\n{list(df.columns)[:30]}"
        )

    return norm_features


def get_feature_weights(model, feature_list, df):
    """
    Return a sorted Series of feature weights for a fitted pipeline model.
    """
    norm_features = normalize_feature_list(feature_list, df)
    logreg = model.named_steps["logisticregression"]
    coefs = logreg.coef_[0]  # shape (n_features,)

    weights = pd.Series(coefs, index=norm_features).sort_values(ascending=False)
    intercept = float(logreg.intercept_[0])
    return weights, intercept


# ---- For your two models ----
nofseek_weights, nofseek_intercept = get_feature_weights(
    nofseek_model, nofs_opt_feat, results
)
fseek_weights, fseek_intercept = get_feature_weights(
    fseek_model, fs_opt_feat, results
)

print("=== NoFoldseek model feature weights ===")
print(nofseek_weights)
print("Intercept:", nofseek_intercept)

print("\n=== Foldseek model feature weights ===")
print(fseek_weights)
print("Intercept:", fseek_intercept)


=== NoFoldseek model feature weights ===
avg_iptm               1.610355
best_plddt_max         0.292402
num_unique_contacts    0.081868
best_pae_min          -0.244378
max_iptm              -0.351177
dtype: float64
Intercept: 0.09456490639862897

=== Foldseek model feature weights ===
hm_frac_tm0.8             1.196562
best_plddt_max            0.735575
hm_frac_tm0.9             0.407898
multimer_frac_tm0.6       0.323644
structural_consensus      0.225121
total_interaction_area    0.213918
best_pae_min             -0.602685
multimer_frac_tm0.8      -0.891486
dtype: float64
Intercept: 0.16108041276809557


## Save final models

In [7]:
import joblib

joblib.dump(
    {
        "model": nofseek_model,
        "features": nofs_opt_feat,
    },
    "nofseek_logreg.joblib",
)

joblib.dump(
    {
        "model": fseek_model,
        "features": fs_opt_feat,
    },
    "fseek_logreg.joblib",
)


['fseek_logreg.joblib']

## Load Saved Models

In [ ]:
import joblib

function_dir = "" # specify where the files are located
nofseek_bundle = joblib.load(function_dir+"nofseek_logreg.joblib")
nofseek_model_loaded = nofseek_bundle["model"]
nofs_opt_feat_loaded = nofseek_bundle["features"]

fseek_bundle = joblib.load(function_dir+"fseek_logreg.joblib")
fseek_model_loaded = fseek_bundle["model"]
fs_opt_feat_loaded = fseek_bundle["features"]

In [13]:
# Use saved models, here using the same data again just to show usage: 

nofs_features_norm = normalize_feature_list(nofs_opt_feat_loaded, results)
fs_features_norm   = normalize_feature_list(fs_opt_feat_loaded, results)

# ⬇️ IMPORTANT: no .values here, keep as DataFrame
X_nofs = results[nofs_features_norm]
X_fs   = results[fs_features_norm]

proba_nofs = nofseek_model_loaded.predict_proba(X_nofs)[:, 1]
proba_fs   = fseek_model_loaded.predict_proba(X_fs)[:, 1]
